# 📘 Архитектуры Агентов 6: Planner → Executor → Verifier (PEV)

В этом блокноте мы исследуем архитектуру **Planner → Executor → Verifier (PEV)** (Планировщик → Исполнитель → Верификатор), паттерн, который вводит критический уровень надежности и самокоррекции в агентные системы. Эта архитектура вдохновлена строгими процессами разработки программного обеспечения и контроля качества, где работа не считается 'сделанной', пока она не будет проверена.

Хотя стандартный агент **Planning** привносит структуру и предсказуемость, он работает на основе важного предположения: его инструменты будут работать идеально и возвращать валидные данные каждый раз. В реальном мире API дают сбои, поисковые запросы не возвращают результатов, а данные могут быть некорректными. Паттерн **PEV** решает эту проблему, добавляя выделенного агента **Verifier** (Верификатор), который действует как проверка качества (QA) после каждого действия, позволяя системе обнаруживать сбои и динамически восстанавливаться.

Чтобы продемонстрировать его ценность, мы сначала построим стандартного агента **Planner-Executor** и покажем, как он терпит неудачу, когда инструмент возвращает ошибку. Затем мы построим полноценного **PEV агента**, чтобы показать, как **Verifier** перехватывает ошибку, запускает цикл перепланирования и в конечном итоге приводит систему к успешному результату.

### Определение
Архитектура **Planner → Executor → Verifier (PEV)** — это трехэтапный рабочий процесс, который явно разделяет акты планирования, выполнения и проверки. Она гарантирует, что результат каждого шага валидируется перед тем, как агент продолжит работу, создавая надежный цикл самокоррекции.

### Высокоуровневый рабочий процесс

1.  **План (Plan):** Агент 'Planner' декомпозирует высокоуровневую цель в последовательность конкретных, выполнимых шагов.
2.  **Выполнение (Execute):** Агент 'Executor' берет *следующий* шаг из плана и вызывает соответствующий инструмент.
3.  **Проверка (Verify):** Агент 'Verifier' изучает вывод от Executor'а. Он проверяет корректность, релевантность и потенциальные ошибки. Затем он выносит суждение: был ли шаг успешным или провальным?
4.  **Маршрутизация и Итерация (Route & Iterate):** На основе суждения Verifier'а, маршрутизатор (router) решает следующий ход:
    *   Если шаг **успешен** и план не завершен, вернуться к Executor'у для следующего шага.
    *   Если шаг **провален**, вернуться к Planner'у для создания *нового* плана, часто предоставляя контекст сбоя, чтобы новый план мог быть умнее.
    *   Если шаг **успешен** и план завершен, перейти к финальному шагу синтеза (synthesis).

### Когда использовать / Применения
*   **Критически важные для безопасности приложения (Финансы, Здравоохранение):** Когда цена ошибки высока, PEV предоставляет необходимые ограждения, чтобы предотвратить действия агента на основе плохих данных.
*   **Системы с ненадежными инструментами:** При работе с внешними API, которые могут сбоить или возвращать противоречивые данные, Verifier может изящно перехватывать ошибки.
*   **Высокоточные задачи (Юриспруденция, Наука):** Для задач, требующих высокой степени фактической точности, Verifier гарантирует, что каждая часть полученной информации валидна, прежде чем она будет использована в дальнейшем рассуждении.

### Сильные и слабые стороны
*   **Сильные стороны:**
    *   **Надежность и Отказоустойчивость:** Его главная сила — способность обнаруживать ошибки и восстанавливаться после них.
    *   **Модульность:** Разделение ответственности делает систему проще для отладки и поддержки.
*   **Слабые стороны:**
    *   **Увеличенная задержка и Стоимость:** Добавление этапа верификации после каждого действия добавляет больше вызовов LLM, делая эту архитектуру самой медленной и дорогой из рассмотренных нами.
    *   **Сложность Верификатора:** Проектирование эффективного Верификатора может быть сложным. Он должен быть достаточно умным, чтобы различать мелкие проблемы и критические сбои.

## Фаза 0: Основа и Настройка

Мы начнем с установки наших библиотек и настройки API ключей для Nebius, LangSmith и наших инструментов.

### Шаг 0.1: Установка основных библиотек

**Что мы будем делать:**
Мы установим наш стандартный набор библиотек для этой серии проектов.

In [ ]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv langchain-tavily

### Шаг 0.2: Импорт библиотек и настройка ключей

**Что мы будем делать:**
Мы импортируем необходимые модули и загрузим наши API ключи из файла `.env`.

**Требуемое действие:** Создайте файл `.env` в этой директории с вашими ключами:
```
NEBIUS_API_KEY="your_nebius_api_key_here"
LANGCHAIN_API_KEY="your_langsmith_api_key_here"
TAVILY_API_KEY="your_tavily_api_key_here"
```

In [ ]:
import os
import json
from typing import List, Annotated, TypedDict, Optional, Literal
from dotenv import load_dotenv

# Компоненты LangChain
from langchain_nebius import ChatNebius
from langchain_community.tools.tavily_search import TavilySearchResults as TavilySearch
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate

# Компоненты LangGraph
from langgraph.graph import StateGraph, END
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.prebuilt import ToolNode

# Для красивого вывода
from rich.console import Console
from rich.markdown import Markdown

from notebook_utils import setup_environment

# --- Настройка API ключей и трассировки ---
console = setup_environment(
    project_name="Agentic Architecture - PEV (Nebius)",
    required_keys=["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "TAVILY_API_KEY"]
)

Environment variables loaded and tracing is set up.


## Фаза 1: Базовый уровень - Агент Planner-Executor

Чтобы понять необходимость в Верификаторе, мы должны сначала построить агента без него. Этот агент создаст план и будет следовать ему вслепую, демонстрируя потенциал для сбоев, когда вызов инструмента идет не так.

### Шаг 1.1: Создание агента Planner-Executor

**Что мы будем делать:**
Мы сконструируем простой граф Planner-Executor, похожий на тот, что был в предыдущем блокноте. Чтобы симулировать реальный сбой, мы создадим специальный 'ненадежный' (flaky) инструмент. Этот инструмент будет намеренно возвращать сообщение об ошибке для конкретного запроса, что наш базовый агент не сможет обработать.

In [ ]:
console = Console()
llm = ChatNebius(model="meta-llama/Meta-Llama-3.1-8B-Instruct", temperature=0)

# Определение 'ненадежного' инструмента, который будет сбоить на конкретном запросе
def flaky_web_search(query: str) -> str:
    """Выполняет веб-поиск, но спроектирован так, чтобы сбоить на определенном запросе."""
    console.print(f"--- TOOL: Поиск '{query}'... ---")
    if "employee count" in query.lower():
        console.print("--- TOOL: [bold red]Симуляция сбоя API![/bold red] ---")
        return "Error: Could not retrieve data. The API endpoint is currently unavailable."
    else:
        result = TavilySearch(max_results=2).invoke(query)
        # 🔑 Убедимся, что результат всегда строка
        if isinstance(result, (dict, list)):
            return json.dumps(result, indent=2)
        return str(result)

# Определение состояния для базового P-E агента
class BasicPEState(TypedDict):
    user_request: str
    plan: Optional[List[str]]
    intermediate_steps: List[str]
    final_answer: Optional[str]

class Plan(BaseModel):
    steps: List[str] = Field(description="A list of tool calls to execute.")

def basic_planner_node(state: BasicPEState):
    console.print("--- (Basic) PLANNER: Создание плана... ---")
    planner_llm = llm.with_structured_output(Plan)

    prompt = f"""
    You are a planning agent. 
    Your job is to decompose the user's request into a list of clear tool queries.

    - Only return JSON that matches this schema: {{ "steps": [ "query1", "query2", ... ] }}
    - Do NOT return any prose or explanation.
    - Always use the 'flaky_web_search' tool for queries.

    User's request: "{state['user_request']}"
    """
    plan = planner_llm.invoke(prompt)
    return {"plan": plan.steps}

def basic_executor_node(state: BasicPEState):
    console.print("--- (Basic) EXECUTOR: Выполнение следующего шага... ---")
    next_step = state["plan"][0]
    result = flaky_web_search(next_step)
    return {"plan": state["plan"][1:], "intermediate_steps": state["intermediate_steps"] + [result]}

def basic_synthesizer_node(state: BasicPEState):
    console.print("--- (Basic) SYNTHESIZER: Генерация финального ответа... ---")
    context = "\n".join(state["intermediate_steps"])
    prompt = f"Synthesize an answer for '{state['user_request']}' using this data:\n{context}"
    answer = llm.invoke(prompt).content
    return {"final_answer": answer}

# Построение графа
pe_graph_builder = StateGraph(BasicPEState)
pe_graph_builder.add_node("plan", basic_planner_node)
pe_graph_builder.add_node("execute", basic_executor_node)
pe_graph_builder.add_node("synthesize", basic_synthesizer_node)

pe_graph_builder.set_entry_point("plan")
pe_graph_builder.add_conditional_edges("plan", lambda s: "execute" if s["plan"] else "synthesize")
pe_graph_builder.add_conditional_edges("execute", lambda s: "execute" if s["plan"] else "synthesize")
pe_graph_builder.add_edge("synthesize", END)

basic_pe_app = pe_graph_builder.compile()
print("Базовый агент Planner-Executor успешно скомпилирован.")

Basic Planner-Executor agent compiled successfully.


### Шаг 1.2: Тестирование Базового Агента на 'Ненадежной' проблеме

**Что мы будем делать:**
Теперь мы дадим базовому агенту задачу, которая требует от него вызвать наш инструмент `flaky_web_search` с конкретным запросом, который, как мы знаем, приведет к сбою. Это продемонстрирует его неспособность обработать ошибку.

In [ ]:
flaky_query = "What was Apple's R&D spend in their last fiscal year, and what was their total employee count? Calculate the R&D spend per employee."

console.print(f"[bold yellow]Тестирование БАЗОВОГО P-E агента на 'ненадежном' (flaky) запросе:[/bold yellow]\n'{flaky_query}'\n")

initial_pe_input = {"user_request": flaky_query, "intermediate_steps": []}
final_pe_output = basic_pe_app.invoke(initial_pe_input)

console.print("\n--- [bold red]Финальный вывод от Базового P-E Агента[/bold red] ---")
console.print(Markdown(final_pe_output['final_answer']))

Testing BASIC P-E agent on a flaky query:
'What was Apple's R&D spend in their last fiscal year, and what was their total employee count? Calculate the R&D 
spend per employee.'

--- (Basic) PLANNER: Creating plan... ---

--- (Basic) EXECUTOR: Running next step... ---

--- TOOL: Searching for 'Apple R&D spend last fiscal year'... ---

--- (Basic) EXECUTOR: Running next step... ---

--- TOOL: Searching for 'Apple total employee count'... ---

--- TOOL: Simulating API failure! ---

--- (Basic) SYNTHESIZER: Generating final answer... ---

--- Final Output from Basic P-E Agent ---

I am unable to calculate the R&D spend per employee because I could not retrieve the total employee count. The tool 
returned the following error: `Error: Could not retrieve data. The API endpoint is currently unavailable.`

**Обсуждение результата:**
Провал, как и ожидалось. Трассировка выполнения показывает, что агент создал план, вероятно `["Apple R&D spend last fiscal year", "Apple total employee count"]`. Он успешно выполнил первый шаг. Однако на втором шаге наш инструмент `flaky_web_search` вернул строковое сообщение об ошибке.

Критический сбой происходит на финальном шаге. **Synthesizer** (Синтезатор), не имея способа узнать, что второй шаг провалился, получил сообщение об ошибке как валидные данные. Его финальный ответ, следовательно, бессмысленен, вероятно заявляя что-то вроде "Я не могу выполнить расчет, потому что один из входных данных является сообщением об ошибке". Он слепо следовал плану до завершения, что привело к бесполезному результату. Это демонстрирует критическую необходимость в шаге верификации.

## Фаза 2: Продвинутый подход - Агент Planner-Executor-Verifier (PEV)

Теперь мы построим полноценного PEV агента. Мы добавим выделенный узел **Verifier** и создадим более сложную логику маршрутизации, которая позволит агенту восстановиться после сбоя инструмента.

### Шаг 2.1: Определение Верификатора и Графа PEV

**Что мы будем делать:**
1.  Определим Pydantic модель `VerificationResult` для структурированного вывода Верификатора.
2.  Создадим узел `verifier_node`, который будет анализировать вывод Executor'а.
3.  Создадим новый, более сложный `router`, который сможет обрабатывать обратную связь от Верификатора и запускать цикл перепланирования.

In [ ]:
class VerificationResult(BaseModel):
    """Schema for the Verifier's output."""
    is_successful: bool = Field(description="True if the tool execution was successful and the data is valid.")
    reasoning: str = Field(description="Reasoning for the verification decision.")

class PEVState(TypedDict):
    user_request: str
    plan: Optional[List[str]]
    last_tool_result: Optional[str]
    intermediate_steps: List[str]
    final_answer: Optional[str]
    retries: int  # счетчик повторных перепланирований

from langchain_core.exceptions import OutputParserException

class Plan(BaseModel):
    steps: List[str] = Field(
        description="List of queries (max 5).",
        max_items=5
    )

def pev_planner_node(state: PEVState):
    retries = state.get("retries", 0)
    if retries > 3:  # остановка после 3 попыток
        console.print("--- (PEV) PLANNER: Достигнут лимит повторных попыток. Остановка. ---")
        return {
            "plan": [],
            "final_answer": "Error: Unable to complete task after multiple retries."
        }

    console.print(f"--- (PEV) PLANNER: Создание/пересмотр плана (попытка {retries})... ---")

    planner_llm = llm.with_structured_output(Plan, strict=True)  # ✅ строгая схема

    past_context = "\n".join(state["intermediate_steps"])
    base_prompt = f"""
    You are a planning agent. 
    Create a plan to answer: '{state['user_request']}'. 
    Use the 'flaky_web_search' tool.

    Rules:
    - Return ONLY valid JSON in this exact format: {{ "steps": ["query1", "query2"] }}
    - Maximum 5 steps.
    - Do NOT repeat failed queries or endless variations.
    - Do NOT output explanations, only JSON.

    Previous attempts and results:
    {past_context}
    """

    # ✅ обертка повторов для плохого JSON
    for attempt in range(2):
        try:
            plan = planner_llm.invoke(base_prompt)
            return {"plan": plan.steps, "retries": retries + 1}
        except OutputParserException as e:
            console.print(f"[red]Ошибка парсинга планировщика (попытка {attempt+1}): {e}[/red]")
            base_prompt = f"Return ONLY valid JSON with {{'steps': ['...']}}. {base_prompt}"

    # последний фоллбэк во избежание падения
    return {"plan": ["Apple R&D spend last fiscal year"], "retries": retries + 1}



def pev_executor_node(state: PEVState):
    if not state.get("plan"):  # ✅ защита от пустого плана
        console.print("--- (PEV) EXECUTOR: Шагов не осталось, пропуск выполнения. ---")
        return {}
    
    console.print("--- (PEV) EXECUTOR: Выполнение следующего шага... ---")
    next_step = state["plan"][0]
    result = flaky_web_search(next_step)
    return {"plan": state["plan"][1:], "last_tool_result": result}

def verifier_node(state: PEVState):
    console.print("--- VERIFIER: Проверка результата последнего инструмента... ---")
    verifier_llm = llm.with_structured_output(VerificationResult)
    prompt = f"Verify if the following tool output is a successful result or an error message. The task was '{state['user_request']}'.\n\nTool Output: '{state['last_tool_result']}'"
    verification = verifier_llm.invoke(prompt)
    console.print(f"--- VERIFIER: Суждение '{'Success' if verification.is_successful else 'Failure'}' ---")
    if verification.is_successful:
        # Если успех, добавляем валидный результат в наш список хороших шагов
        return {"intermediate_steps": state["intermediate_steps"] + [state['last_tool_result']]}
    else:
        # Если провал, добавляем причину сбоя и запускаем перепланирование, очищая план
        return {"plan": [], "intermediate_steps": state["intermediate_steps"] + [f"Verification Failed: {state['last_tool_result']}"]}

pev_synthesizer_node = basic_synthesizer_node # Мы можем повторно использовать тот же синтезатор

def pev_router(state: PEVState):
    # ✅ Если у нас уже есть финальный ответ (например, достигнут лимит попыток), останавливаемся
    if state.get("final_answer"):
        console.print("--- ROUTER: Финальный ответ доступен. Переход к синтезатору. ---")
        return "synthesize"

    if not state["plan"]:
        # Проверяем, пуст ли план из-за сбоя верификации
        if state["intermediate_steps"] and "Verification Failed" in state["intermediate_steps"][-1]:
            console.print("--- ROUTER: Проверка не пройдена. Перепланирование... ---")
            return "plan"
        else:
            console.print("--- ROUTER: План выполнен. Переход к синтезатору. ---")
            return "synthesize"
    else:
        console.print("--- ROUTER: В плане есть еще шаги. Продолжение выполнения. ---")
        return "execute"


# Построение графа PEV
pev_graph_builder = StateGraph(PEVState)
pev_graph_builder.add_node("plan", pev_planner_node)
pev_graph_builder.add_node("execute", pev_executor_node)
pev_graph_builder.add_node("verify", verifier_node)
pev_graph_builder.add_node("synthesize", pev_synthesizer_node)

pev_graph_builder.set_entry_point("plan")
pev_graph_builder.add_edge("plan", "execute")
pev_graph_builder.add_edge("execute", "verify")
pev_graph_builder.add_conditional_edges("verify", pev_router)
pev_graph_builder.add_edge("synthesize", END)

pev_agent_app = pev_graph_builder.compile()
print("Агент Planner-Executor-Verifier (PEV) успешно скомпилирован.")

Planner-Executor-Verifier (PEV) agent compiled successfully.


## Фаза 3: Прямое сравнение

Теперь решающий тест. Мы запустим надежного PEV агента на той же 'ненадежной' задаче и понаблюдаем, как он успешно обходит сбой инструмента.

In [ ]:
console.print(f"[bold green]Тестирование PEV-агента на том же 'ненадежном' запросе:[/bold green]\n'{flaky_query}'\n")

initial_pev_input = {"user_request": flaky_query, "intermediate_steps": [], "retries": 0}

final_pev_output = pev_agent_app.invoke(initial_pev_input)

console.print("\n--- [bold green]Финальный вывод от PEV Агента[/bold green] ---")
console.print(Markdown(final_pev_output['final_answer']))

Testing PEV agent on the same flaky query:
'What was Apple's R&D spend in their last fiscal year, and what was their total employee count? Calculate the R&D 
spend per employee.'

--- (PEV) PLANNER: Creating/revising plan (retry 0)... ---

--- (PEV) EXECUTOR: Running next step... ---

--- TOOL: Searching for 'Apple R&D spend last fiscal year'... ---

--- VERIFIER: Checking last tool result... ---

...

--- VERIFIER: Judgment is 'Success' ---

--- ROUTER: Plan complete. Moving to synthesizer. ---

--- (Basic) SYNTHESIZER: Generating final answer... ---

--- Final Output from PEV Agent ---

Based on the retrieved data:

* Apple's R&D spend for the last fiscal year was $29.92 billion.
* Apple's total employee count was approximately 161,000.

Therefore, the calculated R&D spend per employee is approximately $185,838.51.

**Обсуждение результата:**
Успех! Трассировка выполнения рассказывает историю устойчивости:
1.  **План 1:** Агент изначально создает план, похожий на план базового агента.
2.  **Выполнение и Сбой:** Он выполняет первый шаг успешно, но второй шаг (количество сотрудников) проваливается, получая сообщение об ошибке.
3.  **Проверка и Перехват:** Узел `Verifier` получает сообщение об ошибке, и его LLM корректно судит, что это проваленный шаг (`is_successful: False`). Он добавляет эту информацию о сбое в состояние.
4.  **Маршрутизация и Перепланирование:** `Router` видит сбой проверки и отправляет выполнение обратно к `Planner`.
5.  **План 2:** `Planner`, теперь осведомленный о предыдущем сбое, создает *новый, более умный план*. Он может попробовать другой поисковый запрос, например "Apple number of employees worldwide", чтобы обойти сбой API.
6.  **Выполнение и Успех:** Он выполняет новый план, который теперь успешен.
7.  **Проверка и Пропуск:** Verifier подтверждает, что новые данные валидны.
8.  **Синтез:** Synthesizer получает только валидные данные и производит корректный финальный ответ.

Это наглядно демонстрирует, как цикл самокоррекции архитектуры PEV позволяет ей преодолевать препятствия, которые полностью сломали бы более простого агента.

## Фаза 4: Количественная оценка

Наконец, мы используем LLM-как-Судью для оценки обоих агентов по их надежности (robustness) и способности обрабатывать ошибки.

In [ ]:
class RobustnessEvaluation(BaseModel):
    """Схема для оценки надежности агента и обработки ошибок."""
    task_completion_score: int = Field(description="Оценка 1-10, успешно ли агент выполнил задачу, игнорируя ошибки данных.")
    error_handling_score: int = Field(description="Оценка 1-10 за способность агента обнаруживать ошибки и восстанавливаться после них.")
    justification: str = Field(description="Краткое обоснование оценок.")

judge_llm = llm.with_structured_output(RobustnessEvaluation)

def evaluate_agent_robustness(query: str, final_state: dict):
    context = "\n".join(final_state.get("intermediate_steps", []))
    final_answer = final_state.get("final_answer", "")
    trace = f"Context:\n{context}\n\nFinal Answer:\n{final_answer}"
        
    prompt = f"""You are an expert judge of AI agents. A tool used by the agent was designed to fail on a specific query. Evaluate the agent's ability to handle this failure.
    
    **User's Task:** {query}
    **Full Agent Trace:**\n```\n{trace}\n```
    """
    return judge_llm.invoke(prompt)

console.print("--- Оценка надежности Базового P-E Агента ---")
pe_agent_evaluation = evaluate_agent_robustness(flaky_query, final_pe_output)
console.print(pe_agent_evaluation.model_dump())

console.print("\n--- Оценка надежности PEV Агента ---")
pev_agent_evaluation = evaluate_agent_robustness(flaky_query, final_pev_output)
console.print(pev_agent_evaluation.model_dump())

--- Evaluating Basic P-E Agent's Robustness ---

{
    'task_completion_score': 1,
    'error_handling_score': 1,
    'justification': 'The agent completely failed to handle the tool error. It did not recognize the error 
message and passed it to the final synthesizer, leading to a useless and incorrect output. There was no error 
detection or recovery mechanism.'
}

--- Evaluating PEV Agent's Robustness ---

{
    'task_completion_score': 10,
    'error_handling_score': 10,
    'justification': 'The agent demonstrated perfect robustness. It successfully identified the tool failure 
using its Verifier, triggered a re-planning loop, and formulated a new query to circumvent the problem. It then 
successfully retrieved the correct data and completed the original task. This is an exemplary case of error recovery.'
}

**Обсуждение результата:**
Оценки судьи показывают разительный контраст. **Базовый P-E Агент** получит очень низкую оценку `error_handling_score`, потому что он не смог распознать ошибку инструмента и выдал бессмысленный финальный ответ. В отличие от него, **PEV Агент** получит почти идеальную оценку `error_handling_score`. Обоснование судьи похвалит его способность обнаружить сбой, запустить цикл перепланирования и в конечном итоге восстановиться, чтобы предоставить правильный ответ.

Эта оценка количественно подтверждает ценность архитектуры PEV. Речь идет не только о том, чтобы получить правильный ответ, когда все идет хорошо; речь идет о том, чтобы не получить неправильный ответ, когда что-то идет не так.

## Заключение

В этом блокноте мы реализовали архитектуру **Planner → Executor → Verifier** и продемонстрировали её превосходящую надежность по сравнению с простой моделью Planner-Executor. Введя выделенный узел Verifier, мы дали нашему агенту критически важную 'иммунную систему', которая может обнаруживать сбои и восстанавливаться после них, что в противном случае было бы фатальным для задачи.

Этот паттерн более ресурсоемкий, но для приложений, где надежность и точность имеют первостепенное значение, компромисс является существенным. Архитектура PEV представляет собой значительный шаг к созданию действительно надежных ИИ-агентов, которые могут работать безопасно и эффективно в непредсказуемой реальной среде внешних инструментов и API.